<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/Hippo_Seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries


In [1]:
# Purpose: Import necessary libraries for data processing, model building, and visualization.
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from skimage.io import imread, imshow
from skimage.transform import resize
from tqdm import tqdm
from sklearn.utils import shuffle
from tensorflow.keras import backend as K
import seaborn as sns
from sklearn.metrics import confusion_matrix
import shutil
from google.colab import drive
import pandas as pd

# Set random seed for reproducibility
seed = 2019
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Google Drive Setup


In [ ]:
# Purpose: Mount Google Drive, set paths for dataset and models, and copy dataset if not already present.
drive.mount('/content/drive')

# Define Drive paths
drive_dataset_path = '/content/drive/MyDrive/Hippocampus_Dataset'
drive_model_path = '/content/drive/MyDrive/Hippocampus_Models'
local_dataset_path = '/content/hippocampus_dataset'

# Create directories if they don't exist
os.makedirs(drive_dataset_path, exist_ok=True)
os.makedirs(drive_model_path, exist_ok=True)

# Check if dataset exists in Drive
if not os.path.exists(os.path.join(drive_dataset_path, 'aug')):
    print("Copying dataset to Google Drive...")
    shutil.copy('/content/hippocampus segmentation dataset.zip', drive_dataset_path)
    !unzip -q "{os.path.join(drive_dataset_path, 'hippocampus segmentation dataset.zip')}" -d "{local_dataset_path}"
    shutil.copytree(local_dataset_path, os.path.join(drive_dataset_path, 'hippocampus_dataset'))
else:
    print("Loading dataset from Google Drive...")
    shutil.copytree(os.path.join(drive_dataset_path, 'hippocampus_dataset'), local_dataset_path)

# Debug: Check dataset contents
print("Dataset contents:")
!ls {local_dataset_path}
print("Images folder:")
!ls {os.path.join(local_dataset_path, 'aug/images')}
print("Left masks folder:")
!ls {os.path.join(local_dataset_path, 'aug/masks/left')}
print("Right masks folder:")
!ls {os.path.join(local_dataset_path, 'aug/masks/right')}